# Python for Data Science Workshop @VeloCity



## 1. Jupyter Notebook

Jupyter notebook is often used by data scientists who work in Python. It is loosely based on Mathematica and combines code, text and visual output in one page.

Some relevant short cuts:
* ```SHIFT + ENTER``` executes 1 block of code called a cell
* Tab-completion is omnipresent after the import of a package has been executed
* ```SHIFT + TAB``` gives you extra information on what parameters a function takes
* Repeating ```SHIFT + TAB``` multiple times gives you even more information

To get used to these short cuts try them out on the cell below.

In [ ]:
print 'Hello world!'
print range(5)

## 2. Anomaly Detection

### 2.1 Load Data

First we will load the data using a [pickle format](https://docs.python.org/2/library/pickle.html). (We use ```import cPickle as pickle``` because cPickle is faster.)

The data we use contains the pageviews of one of our own websites and for convenience there is only 1 data point per hour.

In [ ]:
import cPickle as pickle

past = pickle.load(open('data/past_data.pickle'))
all_data = pickle.load(open('data/all_data.pickle'))

### 2.2 Plot past data

To plot the past data we will use ```matplotlib.pyplot```. For convenience we import it as ```plt```. 
```% matplotlib inline``` makes sure you can see the output in the notebook. 

(Use ```% matplotlib notebook``` if you want to make it ineractive. Don't forget to click the power button to finish the interaction and to be able to plot a new figure.)

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(20,4)) # This creates a new figure with the dimensions of 20 by 4
plt.plot(past)             # This creates the actual plot
plt.show()                 # This shows the plot

### 2.3 Find the minimum and maximum

Use [```np.nanmax()```](http://docs.scipy.org/doc/numpy/reference/generated/numpy.nanmax.html#numpy.nanmax) and [```np.nanmin()```](http://docs.scipy.org/doc/numpy/reference/generated/numpy.nanmin.html) to find the minmum and maximum while ignoring the NaNs.

In [ ]:
import numpy as np

In [ ]:
maximum = 
minimum = 

print minimum, maximum

And plot these together with the data using the [```plt.axhline()```](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.axhline) function.

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(past)
plt.axhline(maximum, color='r')
plt.axhline(minimum, color='r')
plt.show()

### 2.4 Testing the model on unseen data

Now plot all the data instead of just the past data

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(all_data, color='g')
plt.plot(past, color='b')
plt.axhline(maximum, color='r')
plt.axhline(minimum, color='r')
plt.show()

You can clearly see now  that this model does not detect any anomalies. However, the last day of data clearly looks different compared to the other days.

In what follows we will build a better model for anomaly detection that is able to detect these 'shape shifts' as well.

### 2.5 Building a model with seasonality

To do this we are going to take a step by step approach. Maybe it won't be clear at first why every step is necessary, but that will become clear throughout the process.

First we are going to reshape the past data to a 2 dimensional array with 24 columns. This wil give is 1 row for each day and 1 column for each hour. For this we are going to use the [```np.reshape()```](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html) function. The newshape parameter is a tuple which in this case should be ```(-1, 24)```. If you use a ```-1``` the reshape function will automatically compute that dimension. Pay attention to the order in which the numbers are repositonned (the default ordering should work fine here).

In [ ]:
reshaped_past = 

Now we are going to compute the average over all days. For this we are going to use the [```np.mean()```](https://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html) with the axis variable set to the first dimension (```axis=0```). Next we are going to plot this.

In [ ]:
average_past = 

plt.plot(average_past)
plt.show()

What you can see in the plot above is the average number of pageviews for eacht hour of the day.

Now let's plot this together with the past data on 1 plot. Use a [for loop](https://wiki.python.org/moin/ForLoop) and the [```np.concatenate()```](http://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html) function to concatenate this average 6 times into the variable ```model```.

In [ ]:
model = []

### concatenate here ###

plt.figure(figsize=(20,4))    
plt.plot(model, color='k')
plt.plot(past, color='b')
plt.show()

In the next step we are going to compute the maximum (= positive) and minimum (= negative) deviations from the average to determine what kind of deviations are normal. (Just substract the average/model from the past and take the min and the max of that)

In [ ]:
delta_max = 
delta_min = 

Now let's plot this.

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(model, color='k')
plt.plot(past, color='b')
plt.plot(model + delta_max, color='r')
plt.plot(model + delta_min, color='r')
plt.show()

Now let's test this on all data

In [ ]:
model_all = np.concatenate((model, average_past))

plt.figure(figsize=(20,4))
plt.plot(all_data, color='g')
plt.plot(model_all, color='k')
plt.plot(past, color='b')
plt.plot(model_all + delta_max, color='r')
plt.plot(model_all + delta_min, color='r')
plt.show()

Now you can clearly see where the anomaly is detected by this more advanced model. The code below can gives you the exact indices where an anomaly is detected. The functions uses are the following [```np.where()```](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html) and [```np.logical_or()```](http://docs.scipy.org/doc/numpy/reference/generated/numpy.logical_or.html).

In [ ]:
anomaly_timepoints = np.where(np.logical_or(all_data < model_all + delta_min, all_data > model_all + delta_max))[0]

plt.figure(figsize=(20,4))
plt.scatter(anomaly_timepoints, all_data[anomaly_timepoints], color='r', linewidth=8)
plt.plot(all_data, color='g')
plt.plot(model_all, color='k')
plt.plot(past, color='b')
plt.plot(model_all + delta_max, color='r')
plt.plot(model_all + delta_min, color='r')
plt.xlim(0, len(all_data))
plt.show()

print 'The anomaly occurs at the following timestamps:', anomaly_timepoints

## 3. Modeling

It is often desired to understand the relationship between different sources of information. As an example we'll consider the historical request rate of a web server and compare it to its CPU usage. We'll try to predict the CPU usage of the server based on the request rates of the different pages. First some imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
pylab.rcParams['figure.figsize'] = (13.0, 8.0)
%matplotlib inline

### 3.1 Data import and inspection

[Pandas](http://pandas.pydata.org/) is a popular library for data wrangling, we'll use it to load and inspect a csv file that contains the historical web request and cpu usage of a web server:

In [ ]:
data = pd.DataFrame.from_csv("data/request_rate_vs_CPU.csv")

The head command allows one to quickly see the structure of the loaded data:

In [ ]:
data.head()

We can select the CPU column and plot the data:

In [ ]:
data.plot(figsize=(13,8), y="CPU")

Next we plot the request rates, leaving out the CPU column  as it has another unit:

In [ ]:
data.drop('CPU',1).plot(figsize=(13,8))

Now to continue and start to model the data, we'll work with basic numpy arrays.

We extract the column labels as the request_names for later reference:

In [ ]:
request_names = data.drop('CPU',1).columns.values
request_names

We extract the request rates as a 2-dimensional numpy array:

In [ ]:
request_rates = data.drop('CPU',1).values

and the cpu usage as a one-dimensional numpy array

In [ ]:
cpu = data['CPU'].values

### 3.2 Simple linear regression

First, we're going to work with the total request rate on the server, and compare it to the CPU usage. The numpy function [sum](http://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html) can be used to calculate the total request rate when selecting the right direction (axis) for th summation.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Let's plot the total request rate to check:

In [ ]:
plt.figure(figsize=(13,8))
plt.plot(total_request_rate)

We can make use of a [PyPlot's scatter plot](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.scatter) to understand the relation between the total request rate and the CPU usage:

In [ ]:
# fill in 
plt.figure(figsize=(13,8))
plt.xlabel("Total request rate")
plt.ylabel("CPU usage")
plt.scatter( ...

There clearly is a strong correlation between the request rate and the CPU usage. Now we'll try to capture this relation using a linear model:
$$ \text{cpu} = c_0 + c_1 \text{total_request_rate} $$

For that we'll make use of the [scikit-learn](http://scikit-learn.org/stable/) machine learning library for Python and use [least-squares linear regression](http://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares)

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()

Now we need to feed the data to the model to fit it. The model.fit method expects a matrix so we need to convert the total_request_rate into a matrix with one column, we can 
use the [reshape](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html) method for that:

In [ ]:
# fill in
total_request_rate_M = 

Then we fit our model using the the total request rate and cpu.

In [ ]:
#fill in
model.fit(...

We can now inspect the coefficient $c_1$ of the model:

In [ ]:
model.coef_

And the constant term $c_0$:

In [ ]:
model.intercept_

Once the model is trained we can use it to [predict](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.predict) the outcome for a given input (or array of inputs). 

What is the expected CPU usage when we have 50 requests per second? 

In [ ]:
# fill in 
model.predict(...

Now we plot the linear model together with our data to verify it captures the relationship correctly (the predict method can accept total_request_rate_M at once).

In [ ]:
# fill in
plt.figure(figsize=(15,10))
plt.scatter( ... , ...  , color='black')
plt.plot( ... , ... , color='blue', linewidth=3)
plt.xlabel("Total request rate")
plt.ylabel("CPU usage")

plt.show()

Our model also has a [score](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score) indicating how well the linear model captures the data. A score of 1 means the data is perfectly linear, a score of 0 (or lower) means the data is not linear at all (and it does not make sense to try to model it that way).

In [ ]:
model.score(...

### 3.3 Multiple linear regression

Now we consider the separate request rates again and build a linear model for that. The model we try to fit takes the form:
$$\text{cpu} = c_0 + c_1 \text{request_rate}_1 + c_2 \text{request_rate}_2 + \ldots + c_n \text{request_rate}_n$$
where the $\text{request_rate}_i$'s correspond the our different requests:


In [ ]:
request_names

No we create a new [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) model.

In [ ]:
# fill in
multi_lin_model = 

Next fit the model on the data:

In [ ]:
# fill in
multi_lin_model.fit(

Which request causes most CPU usage, on a per visit basis? ([np.argmax](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html) finds the index of the greatest element in an array)

In [ ]:
#fill in 
heavy_request = 
print heavy_request

If we want to minimize average CPU usage on this server by deviating traffic of one webpage to another server, which page should we choose?  
One way to determine this is by using the multi_lin_model.predict method. Another way is by directly using the regression formula. Some functions that might be useful for this:
- [np.mean](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html) can used to calculate the mean of the values in a matrix
- a * b will calculate the pairwaize product of two vectors

In [ ]:
# fill in
average_rates = 
request_to_move = 
print request_to_move



## 4. Forecasting

For the forecasting we are going to use page views data, very similar to the data used in the anomaly detection section. It is also page view data and contains 1 sample per hour. 

In [ ]:
train_set = pickle.load(open('data/train_set_forecasting.pickle'))

plt.figure(figsize=(20,4))
plt.plot(train_set)
plt.show()

In the graph above you can clearly see that there is a rising trend in the data.

### 4.1 One-step ahead prediction

This forecasting section will describe the one-step ahead prediction. This means in this case that we will only predict the next data point which is in this case the number of pageviews in the next hour.

Now let's first build a model that tries to predict the next data point from the previous one.

In [ ]:
import sklearn
import sklearn.linear_model
import sklearn.gaussian_process

model = sklearn.linear_model.LinearRegression()

# the input X contains all the data except the last data point
X = train_set[ : -1].reshape((-1, 1)) # the reshape is necessary since sklearn requires a 2 dimensional array

# the output y contains all the data except the first data point
y = train_set[1 : ]

# this code fits the model on the train data
model.fit(X, y)

# this score gives you how well it fits on the train set
# higher is better and 1.0 is perfect
print 'The score of the linear model is', model.score(X, y)

As you can see from the score above, the model is not perfect but it seems to get a relatively high score. Now let's make a prediction into the future and plot this.

To predict the datapoint after that we will use the predicted data to make a new prediction. The code below shows how this works for this data set using the linear model you used earlier. Don't forget to fill out the missing code.

In [ ]:
nof_predictions = 100

import copy
# use the last data point as the first input for the predictions
x_test = copy.deepcopy(train_set[-1]) # make a copy to avoid overwriting the training data

prediction = []
for i in range(nof_predictions):
    # predict the next data point
    y_test = model.predict(x_test.reshape((1,1))) # the reshape is necessary since sklearn requires a 2 dimensional array
    
    ##### Complete this part of the code #####
    prediction.append()
    x_test = 
    ##########################################
    

prediction = np.array(prediction)

plt.figure(figsize=(20,4))
plt.plot(np.concatenate((train_set, prediction[:,0])), 'g')
plt.plot(train_set, 'b')
plt.show()

As you can see from the image above the model doesn't quite seem to fit the data well. Let's see how we can improve this.

### 4.2 Multiple features

If your model is not smart enough there is a simple trick in machine learning to make your model more intelligent (but also more complex). This is by adding more features.

To make our model better we will use more than 1 sample from the past. To make your life easier there is a simple function below that will create a data set for you. The ```width``` parameter sets the number of hours in the past that will be used.

In [ ]:
def convert_time_series_to_Xy(ts, width):
    X, y = [], []
    for i in range(len(ts) - width - 1):
        X.append(ts[i : i + width])
        y.append(ts[i + width])
    return np.array(X), np.array(y)

In [ ]:
width = 5
X, y = convert_time_series_to_Xy(train_set, width)

print X.shape, y.shape

As you can see from the print above both X and y contains 303 datapoints. For X you see that there are now 5 features which contain the pageviews from the 5 past hours.

So let's have a look what the increase from 1 to 5 features results to.

In [ ]:
width = 5
X, y = convert_time_series_to_Xy(train_set, width)
model = sklearn.linear_model.LinearRegression()
model.fit(X,y)
print 'The score of the linear model with width =', width, 'is', model.score(X, y)

Now change the ```width``` parameter to see if you can get a better score.

### 4.3 Over-fitting

Now execute the code below to see the prediction of this model.

In [ ]:
# this is a helper function to make the predictions
def predict(model, train_set, width, nof_points):
    prediction = []
    # create the input data set for the first predicted output
    # copy the data to make sure the orriginal is not overwritten
    x_test = copy.deepcopy(train_set[-width : ]) 
    for i in range(nof_points):
        # predict only the next data point
        prediction.append(model.predict(x_test.reshape((1, -1))))
        # use the newly predicted data point as input for the next prediction
        x_test[0 : -1] = x_test[1 : ]
        x_test[-1] = prediction[-1]
    return np.array(prediction)

In [ ]:
nof_predictions = 200
prediction = predict(model, train_set, width, nof_predictions)

plt.figure(figsize=(20,4))
plt.plot(np.concatenate((train_set, prediction[:,0])), 'g')
plt.plot(train_set, 'b')
plt.show()

As you can see in the image above the prediction is not what you would expect from a perfect model. What happened is that the model learned the training data by hart without 'understanding' what the data is really about. This fenomenon is called over-fitting and will always occur if you make your model more complex.

Now play with the number again to see if you can find a more sensible width.

In [ ]:
width = 
X, y = convert_time_series_to_Xy(train_set, width)
model = sklearn.linear_model.LinearRegression()
model.fit(X,y)
print 'The score of the linear model with width =', width, 'is', model.score(X, y)

prediction = predict(model, train_set, width, 200)

plt.figure(figsize=(20,4))
plt.plot(np.concatenate((train_set, prediction[:,0])), 'g')
plt.plot(train_set, 'b')
plt.show()

As you will have noticed by now is that it is better to have a non-perfect score which will give you a much better outcome. Now try the same thing for the following models:
* ```sklearn.linear_model.RidgeCV()```
* ```sklearn.linear_model.LassoCV()```
* ```sklearn.gaussian_process.GaussianProcess()```

The first 2 models also estimate the noise that is present in the data to avoid overfitting. ```RidgeCV``` will keep the weights that are found small, but it won't put them to zero. ```LassoCV``` on the other hand will put several weights to 0. Execute ```model.coef_``` to see the actual coefficients that have been found.

```GaussianProcess``` is a non-linear method. This makes this method a lot more complex and therefore it will need significantly less features to be able to learn the data by hart (and thus to over-fit). In many cases however this additional complexity allows to better understand the data. Additionally it has the advantage that it can estimate confidance intervals similar to the red lines used in the anomaly detection.

### 4.4 Automation

What we have done up to now is manually selecting the best outcome based on the test result. This can be considered cheating because you have just created a self-fulfilling profecy. Additionally it is not only cheating it is also hard to find the exact ```width``` that gives the best result by just visually inspecting it. So we need a more objective approach to solve this.

To automate this process you can use a validation set. In this case we will use the last 48 hours of the training set to validate the score and select the best parameter value. This means that we will have to use a subset of the training set to fit the model.

In [ ]:
model_generators = [sklearn.linear_model.LinearRegression, sklearn.linear_model.RidgeCV,
                    sklearn.linear_model.LassoCV, sklearn.gaussian_process.GaussianProcess]
best_score = 0

##### Complete this part of the code #####
for model_gen in :
    for width in range(): 
##########################################        
        X, y = convert_time_series_to_Xy(train_set, width)
        # train the model on the first 48 hours
        X_train, y_train = X[ : -48, :], y[ : -48]
        # use the last 48 hours for validation
        X_val, y_val = X[-48 : ], y[-48 : ]
        
        ##### Complete this part of the code #####
        model = 
        ##########################################
        
        # there is a try except clause here because some model do not converge for some data
        try:
            ##### Complete this part of the code #####
            model.fit()
            this_score = model.score()
            ##########################################
            
            if this_score > best_score:
                best_score = this_score
                best_model_gen = model_gen
                best_width = width
        except:
            pass

print best_model_gen().__class__, 'was selected as the best model with a width of', best_width, 'and a validation score of', best_score

If everything is correct the LassoCV methods was selected.

Now we are going to train this best model on all the data. In this way we use all the available data to build a model.

In [ ]:
##### Complete this part of the code #####
width = best_
model = best_
##########################################

X, y = convert_time_series_to_Xy(train_set, width)

##### Complete this part of the code #####
model.fit() # train on the full data set
##########################################

nof_predictions = 200
prediction = predict(model, train_set, width, nof_predictions)

plt.figure(figsize=(20,4))
plt.plot(np.concatenate((train_set, prediction[:,0])), 'g')
plt.plot(train_set, 'b')
plt.show()

Altough the optimal result found here might not be the best visually, it is a far better result than the one you selected manually just because there was no cheating involved ;-).

Some additional info:
* This noise level of ```RidgeCV``` and ```LassoCV``` is estimated by automatically performing train and validation within the method itself. This will make them much more robust against over-fitting. The actual method used is [Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) which is a better approach of what we do here because it repeats the training and validation multiple times for different training and validation sets. The parameter that is set for these methods is often called the regularization parameter in literature and is well suited to avoid over-fitting.
* Although sklearn supports estimating the noise level in Gaussian Processes it is not implemented within the method itself. Newer versions of sklearn seem to entail a lot of changes in this method so possibly it will be integrated in the (near) future. If you want to implement this noise level estimation yourself you can use [their cross-validation tool](http://scikit-learn.org/stable/modules/cross_validation.html) to set the [```alpha``` parameter](http://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor) in version 0.18 of sklearn. (The version used here is 0.17.)



## 5. Competition

@BART mijn inspriratie is op...